## From wiki-pages:


In [1]:
#We import all the necessary packages
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import re
import urllib
import json
import urllib.request as urllib2

First we imported the csv of all songs from Beatles as a Panda DataFrame: 

In [2]:
songs = pd.read_csv("Songs_Sorted_Years.csv") #Song, Year, Month, Day, Lyrics 
DataSongs=pd.DataFrame(songs)


In [4]:
songs_list=list(DataSongs.iloc[:,0])
songs_list_clean=[]
for el in songs_list: 
    el=el.replace('-','_')
    el_song=list(el)
    indexes=[]
    
    for ind, a in enumerate(el_song):
        if ind==0:
            indexes.append(ind) 
        if a=='_':
            indexes.append(ind+1)
            
    indexes_set=set(indexes)
    
    for i in indexes_set:
        el_song[i] = el_song[i].upper()

    string = ''.join(el_song)
    songs_list_clean.append(string)

In [5]:
#WRITER: Lennon&McCartney, McCartney, Lennon, Harrison, Ringo

writer=np.zeros(len(DataSongs))
album=np.zeros(len(DataSongs))
genre=np.zeros(len(DataSongs))

DataSongs['Writer']=writer
DataSongs['Album']=album
DataSongs['Genre']=genre
DataSongs['Song']=songs_list_clean

DataSongs.to_csv('AllAtributes.txt',index=False)


Now, we are going to compare it with the wiki-pages:

In [6]:
baseurl = "https://beatles.fandom.com/api.php?"
action = "action=query"  
content = "prop=revisions&rvprop=content&rvslots=*"
dataformat ="format=json" #instead of json, we can use xml

In [7]:
#Not Run It (Already saved it)
for a in songs_list_clean:
    try:
        title = "titles="+str(a)
        Beatles = "{}{}&{}&{}&{}".format(baseurl, action, content, title,dataformat)
        urllib2.urlretrieve(Beatles , 'wiki_songs/'+str(a)+'.txt')
    except:
        print(a)

In [7]:
pages_path="./wiki_songs/"

for a in songs_list_clean:
    with open(pages_path+a+".txt","r") as f:
        wiki_song= f.read()
        writer= re.findall(r'\[\[Category:Songs written by (.*?)\]\]', wiki_song)
        genre= re.findall(r'genre=(.*?)\\n',wiki_song)
        
        if genre: 
            DataSongs['Genre'][index]= genre[0]

        if writer:
            if writer[0]=="Lennon\\u2013McCartney":
                index=DataSongs.index[DataSongs['Song']==a].tolist()       
                DataSongs['Writer'][index]= 'Lennon/McCartney'
                
            else:
                index=DataSongs.index[DataSongs['Song']==a].tolist() 
                DataSongs['Writer'][index]= writer[0]
                

        else:
            index=DataSongs.index[DataSongs['Song']==a].tolist()       
            DataSongs['Writer'][index]= 'Not found'

/opt/anaconda3/envs/socialGraphs/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/anaconda3/envs/socialGraphs/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [10]:
i=0
refill_songs=[]
for a in songs_list_clean:
    with open(pages_path+a+".txt","r") as f:
        wiki_song= f.read()
        writer= re.findall(r'\[\[Category:Songs written by (.*?)\]\]', wiki_song)
        
        if writer: #82 songs with writer
            i=i+1
            if writer[0]=="Lennon\\u2013McCartney":
                index=DataSongs.index[DataSongs['Song']==a].tolist()       
                DataSongs['Writer'][index]= 'Lennon/McCartney'
                
            else:
                index=DataSongs.index[DataSongs['Song']==a].tolist() 
                DataSongs['Writer'][index]= writer[0]
                
        else: #219 missing
            refill_songs.append(a)
            index=DataSongs.index[DataSongs['Song']==a].tolist()       
            DataSongs['Writer'][index]= 'Not found'
            

refill_songs

['Ps_I_Love_You',
 'Please_Please_Me',
 'Taste_Honey',
 'Anna_Go_Him',
 'Boys',
 'Chains',
 'Do_You_Want_Know_Secret',
 'Theres_Place',
 'Twist_And_Shout',
 'Me_You',
 'Ill_Get_You',
 'Hello_Little_Girl',
 'All_Ive_Got_Do',
 'Devil_Her_Heart',
 'Dont_Bother_Me',
 'It_Wont_Be_Long',
 'Money_Thats_What_I_Want',
 'Not_Second_Time',
 'Please_Mister_Postman',
 'Roll_Over_Beethoven',
 'Till_There_Was_You',
 'You_Wont_See_Me',
 'I_Want_Hold_Your_Hand',
 'Boy',
 'Cry_Shadow',
 'Any_Time_All',
 'Ill_Be_Back',
 'I_Call_Your_Name',
 'Hard_Days_Night',
 'Cant_Buy_Me_Love',
 'Ill_Cry_Instead',
 'You_Know_My_Name',
 'Im_Happy_Just_Dance_You',
 'I_Feel_Fine',
 'Shes_Woman',
 'Babys_Black',
 'Eight_Days_Week',
 'Everybodys_Trying_Be_My_Baby',
 'Honey_Dont',
 'I_Dont_Want_Spoil_Party',
 'Ill_Follow_Sun',
 'Im_Loser',
 'Kansas_City_Hey_Hey_Hey_Hey',
 'Mr_Moonlight',
 'Rock_And_Roll_Music',
 'What_Youre_Doing',
 'Yellow_Submarine',
 'Ticket_Ride',
 'Help',
 'Im_Down',
 'Act_Naturally',
 'Dizzy_Miss_Lizzy

In [ ]:
#for all that "Not found" do something 
for a in songs_list_clean:
    with open(pages_path+a+".txt","r") as f:
        wiki_song= f.read()
        writer= re.findall(r'Written by = \[\[(.*?)\]\]', wiki_song)
        if writer:
            print(a)

### Obtain Album Information 

In [44]:
#Array to save all the Wiki Pages related to albums. 
Albums_pages= ['Sgt._Pepper%27s_Lonely_Hearts_Club_Band_(album)','Let_It_Be_(album)','Hey_Jude_(album)',
               'Help!_(album)','A_Hard_Day%27s_Night_(album)','With_the_Beatles','Please_Please_Me_(album)',
              'Beatles_for_Sale','Yellow_Submarine_Songtrack','Introducing...The_Beatles','Yesterday_and_Today']
Albums_name=['Sgt_Peppers_Lonely_Hearts_Club_Band','Let_It_Be','Hey_Jude','Help!','A_Hard_Days_Night',
             'With_the_Beatles','Please_Please_Me','Beatles_for_Sale','Yellow_Submarine_Songtrack',
            'Introducing...The_Beatles','Yesterday_and_Today']


In [45]:
#Save Album Wiki-Pages
for page in Albums_pages:
    title = "titles="+str(page)
    Album_Page = "{}{}&{}&{}&{}".format(baseurl, action, content, title,dataformat)
    urllib2.urlretrieve(Album_Page  , 'wiki_albums/'+str(page)+'.txt')


https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=Sgt._Pepper%27s_Lonely_Hearts_Club_Band_(album)&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=Let_It_Be_(album)&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=Hey_Jude_(album)&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=Help!_(album)&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=A_Hard_Day%27s_Night_(album)&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=With_the_Beatles&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslots=*&titles=Please_Please_Me_(album)&format=json
https://beatles.fandom.com/api.php?action=query&prop=revisions&rvprop=content&rvslo

In [73]:
pages_path="./wiki_albums/"
artists=['Lennon/McCartney','Paul McCartney|McCartney','Lennon/McCartney|Lennon with McCartney','George Harrison',
         'George Harrison|Harrison','John Lennon','John Lennon|Lennon','Ringo Starr|Starkey','Ringo Starr|Starr',
        'Chuck Berry','Lennon/McCartney|McCartney with Lennon','Lennon/McCartney|McCartney/Lennon',
        'Little Richard|Richard Penniman','George Martin','Vee Jay Records|Vee Jay','Paul McCartney',
        'Ringo Starr|Richard Starkey']

#General Regular Expression for Wiki-Pages of all albums
Tracks='(?:==Tracks==|==Track listing==)'
TotalLength='Total Length'
Credits='(?:Credits|==References==|==American release==|==Personnel==|==External links==)'
pattern_songs_albums="\[\[(.*?)\]\]"

Songs_Albums=[] #Array to store all the Songs from an album
All_Years=[]
All_Albums=[]

for j,page in enumerate(Albums_pages):
    print(page)
    with open(pages_path+page+".txt","r") as f:
        wiki_album= f.read()
        
        #Release Date of the Album
        album_drelease= re.findall(r'release date(.*?)recorded',wiki_album)
        if album_drelease:
            album_yrelease= re.findall(r'\d\d\d\d',album_drelease[0])
        else:
            album_yrelease= re.findall(r'19\d\d',wiki_album)
        
            
        songs=re.findall(r"\[\[(.*?)\]\]",wiki_album)
 

        a=[(m.start(0), m.end(0)) for m in re.finditer(Tracks, wiki_album)]
        if len(a)!=1:
            a=a[1]
        else:
            a=a[0]
        b=[(m.start(0), m.end(0)) for m in re.finditer(TotalLength, wiki_album)]
        if len(b)>0:
            b=b[1]
        else:
            c=[(m.start(0), m.end(0)) for m in re.finditer(Credits, wiki_album)]
            if len(c)>0:
                b=c[0]
            else:
                b=(a[0]+600,a[1]+600)


        ind_songs=[(m.start(0), m.end(0)) for m in re.finditer(pattern_songs_albums, wiki_album)]
        songs=re.findall(r"\[\[(.*?)\]\]",wiki_album)
        
        Songs=[]
        for i,count in enumerate(ind_songs):
            if count[0] > a[1] and count[0] < b[0]:
                if songs[i] not in artists:
                    if re.findall(r"song",songs[i]):
                        r=[(m.start(0), m.end(0)) for m in re.finditer("song", songs[i])]
                        Songs.append(songs[i][0:r[0][0]-1])
                        
                    else:
                        if page=='Introducing...The_Beatles':
                            no_include=['Ask Me Why',"Please Please Me"]
                            
                            if songs[i] not in no_include:
                                Songs.append(songs[i])
                                
                        else:
                            Songs.append(songs[i])
                  
        
        year=np.full(len(Songs), album_yrelease[0])
        All_Years.extend(year)
        album=np.full(len(Songs), Albums_name[j])
        All_Albums.extend(album)
        Songs_Albums.extend(Songs)


Sgt._Pepper%27s_Lonely_Hearts_Club_Band_(album)
Let_It_Be_(album)
Hey_Jude_(album)
Help!_(album)
A_Hard_Day%27s_Night_(album)
With_the_Beatles
Please_Please_Me_(album)
Beatles_for_Sale
Yellow_Submarine_Songtrack
Introducing...The_Beatles
Yesterday_and_Today


In [75]:
DataAlbums= pd.DataFrame(Songs_Albums,columns=['Songs']) 
DataAlbums['Album']=All_Albums
DataAlbums['Release Year']=All_Years

DataAlbums

,Songs,Album,Release Year
0,Sgt. Pepper's Lonely Hearts Club Band,Sgt_Peppers_Lonely_Hearts_Club_Band,1967
1,With a Little Help from My Friends,Sgt_Peppers_Lonely_Hearts_Club_Band,1967
2,Lucy in the Sky with Diamonds,Sgt_Peppers_Lonely_Hearts_Club_Band,1967
3,Getting Better,Sgt_Peppers_Lonely_Hearts_Club_Band,1967
4,Fixing a Hole,Sgt_Peppers_Lonely_Hearts_Club_Band,1967
...,...,...,...
134,And Your Bird Can Sing,Yesterday_and_Today,1966
135,If I Needed Someone,Yesterday_and_Today,1966
136,We Can Work It Out,Yesterday_and_Today,1966
137,What Goes On,Yesterday_and_Today,1966


In [76]:
DataAlbums.to_csv('AllAlbums.txt',index=False)

In [78]:
DataSongs.to_csv('AllAtributes.txt',index=False)